## Cluster code to calculate mean and sliced correlation - up to 15th order

In [3]:
import os
import glob
import numpy as np
from scipy.io import loadmat
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import timecorr as tc
import math
import scipy.spatial.distance as sd
from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce


In [4]:
data_dir = '/dartfs/rc/lab/D/DBIC/CDL/f002s72/timecorr_paper/pieman/results'
corrs_dir = os.path.join(data_dir, 'laplace_50_orderedup_corrs')

## For sliced correlations:

In [ ]:
levels = np.arange(0,16,1)
conditions = ['intact', 'paragraph', 'word', 'rest']

n = .1
percentages = np.arange(n, 1 + n, n)
for l in levels:
    for c in conditions:
        con = os.path.join(corrs_dir, f'lev_{l}'+ f'_{c}' + '.npy')
        print(con)
        try:
            corrs = np.load(con)
            mat_corrs = tc.helpers.vec2mat(corrs)
            sliced_corrs = np.array([])
            next_corrdir = os.path.join(data_dir, 'sliced_corrs', f'level_{l}')
            if not os.path.exists(next_corrdir):
                os.makedirs(next_corrdir)
            for p in percentages:
                s = int(p*np.shape(mat_corrs)[-1])
                slice_corr = mat_corrs[:, :, s-1]
                if sliced_corrs.shape[0] == 0:
                    sliced_corrs = slice_corr
                else:
                    sliced_corrs = np.dstack((sliced_corrs, slice_corr))
            np.save(os.path.join(next_corrdir, f'{c}.npy'), sliced_corrs)
        except:
            print('issue loading: ' + con)
            pass

## For averaged correlations:

In [ ]:
levels = np.arange(0,16,1)
conditions = ['intact', 'paragraph', 'rest', 'word']

for l in levels:
    for c in conditions:
        con = os.path.join(corrs_dir, f'lev_{l}'+ f'_{c}'+ '.npy')
        try:
            corrs = np.load(con)
            mat_corrs = tc.helpers.vec2mat(corrs)
            next_corrdir = os.path.join(data_dir, 'mean_corrs', f'level_{l}')
            if not os.path.exists(next_corrdir):
                os.makedirs(next_corrdir)
            mean_corrs = mat_corrs.mean(axis=2)
            np.save(os.path.join(next_corrdir, f'{c}.npy'), mean_corrs)
        except:
            print('issue loading: ' + con)
            pass